In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import mode

import lightgbm as lgb
from lightgbm import LGBMClassifier

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 10)


In [4]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [5]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6144, 28, 28, 1)


In [6]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6144, 10)


In [7]:
x_test = test[:,1:]
x_test = np.reshape(x_test, (-1, 28, 28, 1))
print(x_test.shape)

(20480, 28, 28, 1)


In [8]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(densenet_type='DenseNet-121', base_growth_rate = 32):
    model_input = Input(shape=(28,28,1))
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    
    return model

In [9]:
def CNN():
    inputs = Input(shape=(28,28,1))
    x = Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    
    return model

In [10]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48]}

DN_model = DenseNet('DenseNet-121', 32)
CNN_model = CNN()

In [11]:
def get_stacking_data(model, x_train, y_train, x_test, n_folds=5):
    kfold = KFold(n_splits = n_folds)
    
    # 최종 모델에서 사용할 데이터셋 셋팅(0 값으로)
    # 만약 shape가 (100, 10) 이었으면 폴드의 검증 과정에서 저장할 데이터는 (100, 1)
    # 모양을 갖게 한다.
    train_fold_predict = np.zeros((x_train.shape[0],1))
    
    # test는 x_test 값을 이용해서 매 폴드마다 예측을 하기 때문에 (100, fold개수)
    # 만큼의 shape를 갖게 된다.
    # 그래서 해당 폴드마다 x_test의 예측 값을 해당 fold에 해당되는 열에 넣는다.
    test_predict = np.zeros((x_test.shape[0], n_folds))
    print("model : ", model.__class__.__name__)
    
    for cnt, (train_index, valid_index) in enumerate(kfold.split(x_train)):
        x_train_ = x_train[train_index]
        y_train_ = y_train[train_index]
        x_validation = x_train[valid_index]
        
        # 학습
        model.fit(x_train_, y_train_, batch_size=64, epochs=30)
        
        # 해당 폴드에서 학습된 모델에다가 검증 데이터(x_validation)로 예측 후 저장
        train_fold_predict[valid_index, :] = np.argmax(model.predict(x_validation), axis=1).reshape(-1,1)
        
        # 해당 폴드에서 생성된 모델에게 원본 테스트 데이터(x_test)를 이용해서 예측을 수행하고 저장
        test_predict[:, cnt] = np.argmax(model.predict(x_test), axis=1)
        print(test_predict)
    
    # for문이 끝나면 test_pred는 가장 많이 나온 값으로 한다.
    test_predict_freq = []
    for i in range(len(test_predict)):
        test_predict_freq.append(int(mode(test_predict[i])[0]))
    test_predict_freq = np.array(test_predict_freq).reshape(-1,1)
    return train_fold_predict, test_predict_freq

In [12]:
DN_train, DN_test = get_stacking_data(DN_model, x_total, y_total, x_test)
CNN_train, CNN_test = get_stacking_data(CNN_model, x_total, y_total, x_test)

model :  Model
Epoch 1/30
4915/4915 [==============================] - 52s 11ms/step - loss: 2.1420 - acc: 0.3089
Epoch 2/30
4915/4915 [==============================] - 27s 6ms/step - loss: 1.1632 - acc: 0.5986
Epoch 3/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.7880 - acc: 0.7288
Epoch 4/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.6372 - acc: 0.7878
Epoch 5/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.5105 - acc: 0.8185
Epoch 6/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.3800 - acc: 0.8684
Epoch 7/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.3127 - acc: 0.8909
Epoch 8/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.2824 - acc: 0.9011
Epoch 9/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.2215 - acc: 0.9211
Epoch 10/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.2544 - acc: 0.9168
Epo

4915/4915 [==============================] - 28s 6ms/step - loss: 0.0055 - acc: 0.9982
Epoch 22/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0142 - acc: 0.9949
Epoch 23/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.0479 - acc: 0.9868
Epoch 24/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0458 - acc: 0.9847
Epoch 25/30
4915/4915 [==============================] - 27s 6ms/step - loss: 0.0826 - acc: 0.9762
Epoch 26/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0841 - acc: 0.9750
Epoch 27/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0506 - acc: 0.9854
Epoch 28/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0427 - acc: 0.9878
Epoch 29/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0199 - acc: 0.9927
Epoch 30/30
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0026 - acc: 0.9988
[[6. 6. 6. 0. 0.]
 [3.

4915/4915 [==============================] - 2s 345us/step - loss: 0.9934 - acc: 0.6596
Epoch 9/30
4915/4915 [==============================] - 2s 343us/step - loss: 0.8917 - acc: 0.6964
Epoch 10/30
4915/4915 [==============================] - 2s 340us/step - loss: 0.8018 - acc: 0.7192
Epoch 11/30
4915/4915 [==============================] - 2s 348us/step - loss: 0.6912 - acc: 0.7648
Epoch 12/30
4915/4915 [==============================] - 2s 341us/step - loss: 0.6177 - acc: 0.7839
Epoch 13/30
4915/4915 [==============================] - 2s 347us/step - loss: 0.5795 - acc: 0.7955 0s - loss: 0.6088
Epoch 14/30
4915/4915 [==============================] - 2s 345us/step - loss: 0.4925 - acc: 0.8309
Epoch 15/30
4915/4915 [==============================] - 2s 347us/step - loss: 0.4028 - acc: 0.8627
Epoch 16/30
4915/4915 [==============================] - 2s 341us/step - loss: 0.3949 - acc: 0.8590
Epoch 17/30
4915/4915 [==============================] - 2s 342us/step - loss: 0.3376 - acc: 0.

4915/4915 [==============================] - 2s 347us/step - loss: 0.0567 - acc: 0.9823 1s - loss: 0
Epoch 28/30
4915/4915 [==============================] - 2s 345us/step - loss: 0.0543 - acc: 0.9819
Epoch 29/30
4915/4915 [==============================] - 2s 342us/step - loss: 0.0546 - acc: 0.9825
Epoch 30/30
4915/4915 [==============================] - 2s 344us/step - loss: 0.0453 - acc: 0.9860
[[6. 6. 6. 0. 0.]
 [9. 9. 3. 0. 0.]
 [8. 8. 8. 0. 0.]
 ...
 [6. 6. 6. 0. 0.]
 [1. 1. 2. 0. 0.]
 [0. 0. 0. 0. 0.]]
Epoch 1/30
4915/4915 [==============================] - 2s 342us/step - loss: 0.0646 - acc: 0.9766
Epoch 2/30
4915/4915 [==============================] - 2s 340us/step - loss: 0.0746 - acc: 0.9756
Epoch 3/30
4915/4915 [==============================] - 2s 343us/step - loss: 0.0680 - acc: 0.9786
Epoch 4/30
4915/4915 [==============================] - 2s 347us/step - loss: 0.0553 - acc: 0.9797
Epoch 5/30
4915/4915 [==============================] - 2s 348us/step - loss: 0.0449 - ac

In [13]:
new_x_train = np.concatenate((DN_train, CNN_train), axis=1)
new_x_test = np.concatenate((DN_test, CNN_test), axis=1)

print("원본 : ", x_total.shape, x_test.shape)
print("새로운 : ", new_x_train.shape, new_x_test.shape)

원본 :  (6144, 28, 28, 1) (20480, 28, 28, 1)
새로운 :  (6144, 2) (20480, 2)


In [14]:
y_train = np.argmax(y_total, axis=1)
lgbm = LGBMClassifier()
model = lgbm.fit(X = new_x_train, y = y_train)

In [15]:
sub = lgbm.predict(new_x_test)
print(sub)

[6 9 8 ... 6 1 0]


In [16]:
#label_test = np.argmax(y_test, axis=1)
#print("정확도 : {0:.4f}".format(accuracy_score(stack_pred, label_test)))

In [19]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = sub
submission.to_csv('data/submission_densenet(0825).csv', index=False)